In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xgboost as xgb

#from sklearn.grid_search import GridSearchCV
#from sklearn.metrics import log_loss
#from sklearn.ensemble import RandomForestClassifier
from IPython.display import display

from Util.data_utils import ReadCSV
from Util.Utils import FirstLook

In [42]:
Path = "D:\\test\\Kaggle\\Util\\dataset\\Instacart"
#aisles, departments, order_products__prior, order_products__train, orders, products, sample_submission = \
#ReadCSV("aisles.csv", "departments.csv", "order_products__prior.csv", "order_products__train.csv", \
#"orders.csv", "products.csv", "sample_submission.csv", path=Path)
order_products__train = \
ReadCSV( "order_products__train.csv", path=Path)

D:\test\Kaggle\Util\dataset\Instacart\order_products__train.csv


In [3]:
orders['order_dom'] = orders['days_since_prior_order']
orders_copy = orders
orders_copy.head(2)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dom
0,2539329,1,prior,1,2,8,NaN,NaN
1,2398795,1,prior,2,3,7,15.0,15.0


In [4]:
orders_copy.order_dom.fillna(1, inplace=True)
orders_copy.head(2)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dom
0,2539329,1,prior,1,2,8,NaN,1.0
1,2398795,1,prior,2,3,7,15.0,15.0


In [5]:
orders_copy['order_dayom'] = orders_copy.groupby(['user_id'])['order_dom'].apply(lambda x:x.cumsum())
display(orders_copy.head(2))

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dom,order_dayom
0,2539329,1,prior,1,2,8,NaN,1.0,1.0
1,2398795,1,prior,2,3,7,15.0,15.0,16.0


In [6]:
del orders_copy['order_dom']
display(orders_copy.head(2))

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom
0,2539329,1,prior,1,2,8,NaN,1.0
1,2398795,1,prior,2,3,7,15.0,16.0


In [7]:
orders_copy['order_dom'] = orders_copy.order_dayom % 30
orders_copy.head(7)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom
0,2539329,1,prior,1,2,8,NaN,1.0,1.0
1,2398795,1,prior,2,3,7,15.0,16.0,16.0
2,473747,1,prior,3,3,12,21.0,37.0,7.0
3,2254736,1,prior,4,4,7,29.0,66.0,6.0
4,431534,1,prior,5,4,15,28.0,94.0,4.0
5,3367565,1,prior,6,2,7,19.0,113.0,23.0
6,550135,1,prior,7,1,9,20.0,133.0,13.0


In [11]:
orders_copy.to_pickle("dataset_1.pickle")

In [4]:
orders_new = pd.read_pickle("dataset_1.pickle")
orders_new.head(5)
orders_copy=orders_new

In [13]:
orders_copy['dow_score'] = pd.Series(np.zeros(orders_copy.shape[0]))

In [22]:
def DOW_Score(df):
    train_dow = df.loc[df['eval_set']=='train', 'order_dow']
    if(train_dow.empty):
        train_dow = df.loc[df['eval_set']=='test', 'order_dow']
    train_dow = train_dow.values
    tmp = df.loc[df['eval_set']=='prior', 'order_dow']
    df.loc[df['eval_set']=='prior', 'dow_score'] = 6-pd.concat([(train_dow-tmp)%7, (tmp-train_dow)%7], axis=1).min(axis=1)
    #display(df)
    return df

orders_copy = orders_copy.groupby(['user_id']).apply(DOW_Score)
orders_copy.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0
2,473747,1,prior,3,3,12,21.0,37.0,7.0,5.0,19.0,25.0
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,6.0,22.0,24.0
4,431534,1,prior,5,4,15,28.0,94.0,4.0,6.0,16.0,22.0


In [20]:
orders_copy['hod_score'] = pd.Series(np.zeros(orders_copy.shape[0]))

def HOD_Score(df):
    train_hod = df.loc[df['eval_set']=='train', 'order_hour_of_day']
    if(train_hod.empty):
        train_hod = df.loc[df['eval_set']=='test', 'order_hour_of_day']
    train_hod = train_hod.values
    tmp = df.loc[df['eval_set']=='prior', 'order_hour_of_day']
    df.loc[df['eval_set']=='prior', 'hod_score'] = 23-pd.concat([(train_hod-tmp)%24, (tmp-train_hod)%24], axis=1).min(axis=1)
    return df

orders_copy = orders_copy.groupby(['user_id']).apply(HOD_Score)
orders_copy.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,21.0,23.0
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,22.0,22.0
2,473747,1,prior,3,3,12,21.0,37.0,7.0,22.0,19.0
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,23.0,22.0
4,431534,1,prior,5,4,15,28.0,94.0,4.0,23.0,16.0


In [21]:
orders_copy['dom_score'] = pd.Series(np.zeros(orders_copy.shape[0]))

def DOM_Score(df):
    train_hod = df.loc[df['eval_set']=='train', 'order_dom']
    if(train_hod.empty):
        train_hod = df.loc[df['eval_set']=='test', 'order_dom']
    train_hod = train_hod.values
    tmp = df.loc[df['eval_set']=='prior', 'order_dom']
    df.loc[df['eval_set']=='prior', 'dom_score'] = 29-pd.concat([(train_hod-tmp)%30, (tmp-train_hod)%30], axis=1).min(axis=1)
    return df

orders_copy = orders_copy.groupby(['user_id']).apply(DOM_Score)
orders_copy.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,21.0,23.0,19.0
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,22.0,22.0,24.0
2,473747,1,prior,3,3,12,21.0,37.0,7.0,22.0,19.0,25.0
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,23.0,22.0,24.0
4,431534,1,prior,5,4,15,28.0,94.0,4.0,23.0,16.0,22.0


In [25]:
orders_copy['orders_score'] = pd.Series(np.zeros(orders_copy.shape[0]))
orders_copy['orders_score'] = orders_copy['order_number'] + ((orders_copy['dow_score']+orders_copy['hod_score']+ \
                                                             orders_copy['dom_score'])/(6+23+29))
orders_copy.head(11)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score,orders_score
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0,1.793103
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0,2.879310
2,473747,1,prior,3,3,12,21.0,37.0,7.0,5.0,19.0,25.0,3.844828
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,6.0,22.0,24.0,4.896552
4,431534,1,prior,5,4,15,28.0,94.0,4.0,6.0,16.0,22.0,5.758621
5,3367565,1,prior,6,2,7,19.0,113.0,23.0,4.0,22.0,17.0,6.741379
6,550135,1,prior,7,1,9,20.0,133.0,13.0,3.0,22.0,27.0,7.896552
7,3108588,1,prior,8,1,14,14.0,147.0,27.0,3.0,17.0,15.0,8.603448
8,2295261,1,prior,9,1,16,0.0,147.0,27.0,3.0,15.0,15.0,9.568966
9,2550362,1,prior,10,4,8,30.0,177.0,27.0,6.0,23.0,15.0,10.758621


In [26]:
orders_copy.to_pickle('dataset_1.pickle')

In [27]:
from Util.Utils import Convert_One_Hot
products_copy = products
products_copy = Convert_One_Hot(products_copy, ['aisle_id'])
products_copy.head(5)

,product_id,product_name,department_id,aisle_id_1,aisle_id_2,aisle_id_3,aisle_id_4,aisle_id_5,aisle_id_6,aisle_id_7,...,aisle_id_125,aisle_id_126,aisle_id_127,aisle_id_128,aisle_id_129,aisle_id_130,aisle_id_131,aisle_id_132,aisle_id_133,aisle_id_134
0,1,Chocolate Sandwich Cookies,19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,All-Seasons Salt,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Robust Golden Unsweetened Oolong Tea,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Green Chile Anytime Sauce,13,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
products_copy['score']=np.zeros(products_copy.shape[0])
products_copy.head(5)

,product_id,product_name,department_id,aisle_id_1,aisle_id_2,aisle_id_3,aisle_id_4,aisle_id_5,aisle_id_6,aisle_id_7,...,aisle_id_126,aisle_id_127,aisle_id_128,aisle_id_129,aisle_id_130,aisle_id_131,aisle_id_132,aisle_id_133,aisle_id_134,score
0,1,Chocolate Sandwich Cookies,19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,2,All-Seasons Salt,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,3,Robust Golden Unsweetened Oolong Tea,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,5,Green Chile Anytime Sauce,13,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [30]:
products_copy.to_pickle('products_1.pickle')

In [2]:
products_copy=pd.read_pickle('products_1.pickle')
orders_copy=pd.read_pickle('dataset_1.pickle')
display(products_copy.head(2))
display(orders_copy.head(2))

,product_id,product_name,department_id,aisle_id_1,aisle_id_2,aisle_id_3,aisle_id_4,aisle_id_5,aisle_id_6,aisle_id_7,...,aisle_id_126,aisle_id_127,aisle_id_128,aisle_id_129,aisle_id_130,aisle_id_131,aisle_id_132,aisle_id_133,aisle_id_134,score
0,1,Chocolate Sandwich Cookies,19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,2,All-Seasons Salt,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score,orders_score
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0,1.793103
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0,2.879310


In [4]:
merged_dataset = pd.merge(order_products__prior, products_copy, on='product_id', how='left' )
merged_dataset.head(10)

,order_id,product_id,add_to_cart_order,reordered,product_name,department_id,aisle_id_1,aisle_id_2,aisle_id_3,aisle_id_4,...,aisle_id_126,aisle_id_127,aisle_id_128,aisle_id_129,aisle_id_130,aisle_id_131,aisle_id_132,aisle_id_133,aisle_id_134,score
0,2,33120,1,1,Organic Egg Whites,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
1,2,28985,2,1,Michigan Organic Kale,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2,2,9327,3,0,Garlic Powder,13,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
3,2,45918,4,1,Coconut Butter,13,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,2,30035,5,0,Natural Sweetener,13,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
5,2,17794,6,1,Carrots,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
6,2,40141,7,1,Original Unflavored Gelatine Mix,13,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
7,2,1819,8,1,All Natural No Stir Creamy Almond Butter,13,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
8,2,43668,9,0,Classic Blend Cole Slaw,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
9,3,33754,1,1,Total 2% with Strawberry Lowfat Greek Strained...,16,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [6]:
column_names = merged_dataset.columns
aisle_columns = [f for f in column_names if 'aisle' in f]
aisle_columns

['aisle_id_1',
 'aisle_id_2',
 'aisle_id_3',
 'aisle_id_4',
 'aisle_id_5',
 'aisle_id_6',
 'aisle_id_7',
 'aisle_id_8',
 'aisle_id_9',
 'aisle_id_10',
 'aisle_id_11',
 'aisle_id_12',
 'aisle_id_13',
 'aisle_id_14',
 'aisle_id_15',
 'aisle_id_16',
 'aisle_id_17',
 'aisle_id_18',
 'aisle_id_19',
 'aisle_id_20',
 'aisle_id_21',
 'aisle_id_22',
 'aisle_id_23',
 'aisle_id_24',
 'aisle_id_25',
 'aisle_id_26',
 'aisle_id_27',
 'aisle_id_28',
 'aisle_id_29',
 'aisle_id_30',
 'aisle_id_31',
 'aisle_id_32',
 'aisle_id_33',
 'aisle_id_34',
 'aisle_id_35',
 'aisle_id_36',
 'aisle_id_37',
 'aisle_id_38',
 'aisle_id_39',
 'aisle_id_40',
 'aisle_id_41',
 'aisle_id_42',
 'aisle_id_43',
 'aisle_id_44',
 'aisle_id_45',
 'aisle_id_46',
 'aisle_id_47',
 'aisle_id_48',
 'aisle_id_49',
 'aisle_id_50',
 'aisle_id_51',
 'aisle_id_52',
 'aisle_id_53',
 'aisle_id_54',
 'aisle_id_55',
 'aisle_id_56',
 'aisle_id_57',
 'aisle_id_58',
 'aisle_id_59',
 'aisle_id_60',
 'aisle_id_61',
 'aisle_id_62',
 'aisle_id_63',
 

In [7]:
from Util.Utils import Delete_Columns
merged_dataset = Delete_Columns(merged_dataset, aisle_columns)

In [8]:
merged_dataset.head(5)

,order_id,product_id,add_to_cart_order,reordered,product_name,department_id,score
0,2,33120,1,1,Organic Egg Whites,16,0.0
1,2,28985,2,1,Michigan Organic Kale,4,0.0
2,2,9327,3,0,Garlic Powder,13,0.0
3,2,45918,4,1,Coconut Butter,13,0.0
4,2,30035,5,0,Natural Sweetener,13,0.0


In [9]:
merged_dataset.to_pickle('merged_1.pickle')

In [15]:
del merged_dataset['score']
merged_dataset.head(4)

,order_id,product_id,add_to_cart_order,reordered,product_name,department_id
0,2,33120,1,1,Organic Egg Whites,16
1,2,28985,2,1,Michigan Organic Kale,4
2,2,9327,3,0,Garlic Powder,13
3,2,45918,4,1,Coconut Butter,13


In [16]:
merged_dataset=pd.merge(merged_dataset, orders_copy.loc[:,['order_id', 'orders_score']], on='order_id', how='left')

,order_id,product_id,add_to_cart_order,reordered,product_name,department_id,orders_score
0,2,33120,1,1,Organic Egg Whites,16,3.810345
1,2,28985,2,1,Michigan Organic Kale,4,3.810345
2,2,9327,3,0,Garlic Powder,13,3.810345
3,2,45918,4,1,Coconut Butter,13,3.810345
4,2,30035,5,0,Natural Sweetener,13,3.810345


In [51]:
merged_dataset.head(15)

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,department_id,orders_score,user_id,products_score,product_count
0,0,2,33120,1,1,16,3.810345,202279,3.810345,1
1,1,2,28985,2,1,4,3.810345,202279,3.810345,1
2,2,2,9327,3,0,13,1.905172,202279,1.905172,1
3,3,2,45918,4,1,13,3.810345,202279,3.810345,1
4,4,2,30035,5,0,13,1.905172,202279,1.905172,1
5,5,2,17794,6,1,4,3.810345,202279,3.810345,1
6,6,2,40141,7,1,13,3.810345,202279,3.810345,1
7,7,2,1819,8,1,13,3.810345,202279,3.810345,1
8,8,2,43668,9,0,4,1.905172,202279,1.905172,1
9,9,3,33754,1,1,16,16.724138,205970,16.724138,1


In [53]:
merged_dataset.loc[:,'orders_score']=merged_dataset.loc[:,'orders_score']*(1-(0.8-0.8*merged_dataset.loc[:,'reordered']))
merged_dataset.head(5)

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,department_id,orders_score,user_id,products_score,product_count
0,0,2,33120,1,1,16,3.810345,202279,3.810345,1
1,1,2,28985,2,1,4,3.810345,202279,3.810345,1
2,2,2,9327,3,0,13,0.381034,202279,1.905172,1
3,3,2,45918,4,1,13,3.810345,202279,3.810345,1
4,4,2,30035,5,0,13,0.381034,202279,1.905172,1


In [20]:
merged_dataset=pd.merge(merged_dataset, orders_copy.loc[:,['order_id', 'user_id']], on='order_id', how='left')
merged_dataset.head(10)

,order_id,product_id,add_to_cart_order,reordered,product_name,department_id,orders_score,user_id
0,2,33120,1,1,Organic Egg Whites,16,3.810345,202279
1,2,28985,2,1,Michigan Organic Kale,4,3.810345,202279
2,2,9327,3,0,Garlic Powder,13,1.905172,202279
3,2,45918,4,1,Coconut Butter,13,3.810345,202279
4,2,30035,5,0,Natural Sweetener,13,1.905172,202279
5,2,17794,6,1,Carrots,4,3.810345,202279
6,2,40141,7,1,Original Unflavored Gelatine Mix,13,3.810345,202279
7,2,1819,8,1,All Natural No Stir Creamy Almond Butter,13,3.810345,202279
8,2,43668,9,0,Classic Blend Cole Slaw,4,1.905172,202279
9,3,33754,1,1,Total 2% with Strawberry Lowfat Greek Strained...,16,16.724138,205970


In [28]:
merged_dataset.head(5)

,order_id,product_id,add_to_cart_order,reordered,department_id,orders_score,user_id
0,2,33120,1,1,16,3.810345,202279
1,2,28985,2,1,4,3.810345,202279
2,2,9327,3,0,13,1.905172,202279
3,2,45918,4,1,13,3.810345,202279
4,2,30035,5,0,13,1.905172,202279


In [33]:
merged_dataset['products_score'] = merged_dataset.loc[:100,:].groupby(['user_id', 'product_id'])['orders_score'].transform('sum')
merged_dataset.head(20)

,order_id,product_id,add_to_cart_order,reordered,department_id,orders_score,user_id,products_score
0,2,33120,1,1,16,3.810345,202279,3.810345
1,2,28985,2,1,4,3.810345,202279,3.810345
2,2,9327,3,0,13,1.905172,202279,1.905172
3,2,45918,4,1,13,3.810345,202279,3.810345
4,2,30035,5,0,13,1.905172,202279,1.905172
5,2,17794,6,1,4,3.810345,202279,3.810345
6,2,40141,7,1,13,3.810345,202279,3.810345
7,2,1819,8,1,13,3.810345,202279,3.810345
8,2,43668,9,0,4,1.905172,202279,1.905172
9,3,33754,1,1,16,16.724138,205970,16.724138


In [40]:
merged_dataset.to_csv('merged_dataset.csv')

In [9]:
merged_dataset = pd.read_csv('merged_dataset.csv')
merged_dataset.head(3)

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,department_id,orders_score,user_id,products_score
0,0,2,33120,1,1,16,3.810345,202279,3.810345
1,1,2,28985,2,1,4,3.810345,202279,3.810345
2,2,2,9327,3,0,13,1.905172,202279,1.905172


In [10]:
merged_dataset[merged_dataset['user_id']==202279]

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,department_id,orders_score,user_id,products_score
0,0,2,33120,1,1,16,3.810345,202279,3.810345
1,1,2,28985,2,1,4,3.810345,202279,3.810345
2,2,2,9327,3,0,13,1.905172,202279,1.905172
3,3,2,45918,4,1,13,3.810345,202279,3.810345
4,4,2,30035,5,0,13,1.905172,202279,1.905172
5,5,2,17794,6,1,4,3.810345,202279,3.810345
6,6,2,40141,7,1,13,3.810345,202279,3.810345
7,7,2,1819,8,1,13,3.810345,202279,3.810345
8,8,2,43668,9,0,4,1.905172,202279,1.905172
990780,990780,104690,1003,1,1,13,8.896552,202279,NaN


In [11]:
merged_dataset.loc[:,'products_score']=merged_dataset.loc[:,'orders_score']
merged_dataset[merged_dataset['user_id']==202279]

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,department_id,orders_score,user_id,products_score
0,0,2,33120,1,1,16,3.810345,202279,3.810345
1,1,2,28985,2,1,4,3.810345,202279,3.810345
2,2,2,9327,3,0,13,1.905172,202279,1.905172
3,3,2,45918,4,1,13,3.810345,202279,3.810345
4,4,2,30035,5,0,13,1.905172,202279,1.905172
5,5,2,17794,6,1,4,3.810345,202279,3.810345
6,6,2,40141,7,1,13,3.810345,202279,3.810345
7,7,2,1819,8,1,13,3.810345,202279,3.810345
8,8,2,43668,9,0,4,1.905172,202279,1.905172
990780,990780,104690,1003,1,1,13,8.896552,202279,8.896552


In [12]:
merged_dataset=merged_dataset.groupby(['user_id', 'product_id']).agg({'products_score':np.sum}).reset_index()

In [13]:
merged_dataset[merged_dataset['user_id']==202279]

,user_id,product_id,products_score
13055970,202279,432,1.370690
13055971,202279,1003,33.146552
13055972,202279,1819,5.181034
13055973,202279,2086,4.000000
13055974,202279,2154,4.448276
13055975,202279,2944,2.456897
13055976,202279,4461,2.913793
13055977,202279,5593,0.922414
13055978,202279,5812,0.922414
13055979,202279,6515,2.913793


In [14]:
merged_dataset.to_pickle('product_score_dataset.pickle')

In [15]:
orders_copy = pd.read_pickle('dataset_1.pickle')
orders_copy.head(4)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score,orders_score
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0,1.793103
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0,2.879310
2,473747,1,prior,3,3,12,21.0,37.0,7.0,5.0,19.0,25.0,3.844828
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,6.0,22.0,24.0,4.896552


In [16]:
merged_dataset = pd.read_csv('merged_dataset.csv')
merged_dataset.head(3)

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,department_id,orders_score,user_id,products_score
0,0,2,33120,1,1,16,3.810345,202279,3.810345
1,1,2,28985,2,1,4,3.810345,202279,3.810345
2,2,2,9327,3,0,13,1.905172,202279,1.905172


In [33]:
merged_dataset['product_count'] = 1
merged_dataset.head(5)
number_products=merged_dataset.groupby(['user_id', 'order_id']).agg({'product_count':np.sum}).reset_index()
number_products.head(10)

,user_id,order_id,product_count
0,1,431534,8
1,1,473747,5
2,1,550135,5
3,1,2254736,5
4,1,2295261,6
5,1,2398795,6
6,1,2539329,5
7,1,2550362,9
8,1,3108588,6
9,1,3367565,4


In [34]:
orders_copy = pd.merge(orders_copy, number_products.loc[:,['order_id', 'product_count']], on='order_id', how='left')
orders_copy.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score,orders_score,product_count
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0,1.793103,5.0
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0,2.879310,6.0
2,473747,1,prior,3,3,12,21.0,37.0,7.0,5.0,19.0,25.0,3.844828,5.0
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,6.0,22.0,24.0,4.896552,5.0
4,431534,1,prior,5,4,15,28.0,94.0,4.0,6.0,16.0,22.0,5.758621,8.0


In [41]:
orders_copy.to_pickle('dataset_1.pickle')

In [36]:
orders_copy['weighted_product_count'] = orders_copy.loc[:,'product_count']*orders_copy.loc[:,'orders_score']

In [37]:
orders_copy.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score,orders_score,product_count,weighted_product_count
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0,1.793103,5.0,8.965517
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0,2.879310,6.0,17.275862
2,473747,1,prior,3,3,12,21.0,37.0,7.0,5.0,19.0,25.0,3.844828,5.0,19.224138
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,6.0,22.0,24.0,4.896552,5.0,24.482759
4,431534,1,prior,5,4,15,28.0,94.0,4.0,6.0,16.0,22.0,5.758621,8.0,46.068966


In [38]:
orders_copy['num_products'] = pd.Series(np.zeros(orders_copy.shape[0]))

def Get_Number_of_Products(df):
    df.loc[:, 'num_products'] = np.sum(df.loc[:, 'weighted_product_count'])/np.sum(df.loc[:, 'orders_score'])
    return df

orders_copy = orders_copy.groupby(['user_id']).apply(Get_Number_of_Products)
orders_copy.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score,orders_score,product_count,weighted_product_count,num_products
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0,1.793103,5.0,8.965517,5.266074
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0,2.879310,6.0,17.275862,5.266074
2,473747,1,prior,3,3,12,21.0,37.0,7.0,5.0,19.0,25.0,3.844828,5.0,19.224138,5.266074
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,6.0,22.0,24.0,4.896552,5.0,24.482759,5.266074
4,431534,1,prior,5,4,15,28.0,94.0,4.0,6.0,16.0,22.0,5.758621,8.0,46.068966,5.266074


In [39]:
orders_copy['num_products']=np.ceil(orders_copy['num_products'])
orders_copy.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score,orders_score,product_count,weighted_product_count,num_products
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0,1.793103,5.0,8.965517,6.0
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0,2.879310,6.0,17.275862,6.0
2,473747,1,prior,3,3,12,21.0,37.0,7.0,5.0,19.0,25.0,3.844828,5.0,19.224138,6.0
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,6.0,22.0,24.0,4.896552,5.0,24.482759,6.0
4,431534,1,prior,5,4,15,28.0,94.0,4.0,6.0,16.0,22.0,5.758621,8.0,46.068966,6.0


In [40]:
orders_copy.head(15)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_dayom,order_dom,dow_score,hod_score,dom_score,orders_score,product_count,weighted_product_count,num_products
0,2539329,1,prior,1,2,8,NaN,1.0,1.0,4.0,23.0,19.0,1.793103,5.0,8.965517,6.0
1,2398795,1,prior,2,3,7,15.0,16.0,16.0,5.0,22.0,24.0,2.879310,6.0,17.275862,6.0
2,473747,1,prior,3,3,12,21.0,37.0,7.0,5.0,19.0,25.0,3.844828,5.0,19.224138,6.0
3,2254736,1,prior,4,4,7,29.0,66.0,6.0,6.0,22.0,24.0,4.896552,5.0,24.482759,6.0
4,431534,1,prior,5,4,15,28.0,94.0,4.0,6.0,16.0,22.0,5.758621,8.0,46.068966,6.0
5,3367565,1,prior,6,2,7,19.0,113.0,23.0,4.0,22.0,17.0,6.741379,4.0,26.965517,6.0
6,550135,1,prior,7,1,9,20.0,133.0,13.0,3.0,22.0,27.0,7.896552,5.0,39.482759,6.0
7,3108588,1,prior,8,1,14,14.0,147.0,27.0,3.0,17.0,15.0,8.603448,6.0,51.620690,6.0
8,2295261,1,prior,9,1,16,0.0,147.0,27.0,3.0,15.0,15.0,9.568966,6.0,57.413793,6.0
9,2550362,1,prior,10,4,8,30.0,177.0,27.0,6.0,23.0,15.0,10.758621,9.0,96.827586,6.0


In [47]:
order_products__train['products_count']=1
order_products__train.head(3)

,order_id,product_id,add_to_cart_order,reordered,products_count
0,1,49302,1,1,1
1,1,11109,2,1,1
2,1,10246,3,0,1


In [48]:
train_product_count = order_products__train.groupby(['order_id']).agg({'products_count':np.sum}).reset_index()
train_product_count.head(5)

,order_id,products_count
0,1,8
1,36,8
2,38,9
3,96,7
4,98,49


In [49]:
train_product_count=pd.merge(train_product_count, orders_copy.loc[:,['order_id', 'num_products']], on='order_id', how='left')
train_product_count.head(5)

,order_id,products_count,num_products
0,1,8,5.0
1,36,8,8.0
2,38,9,7.0
3,96,7,5.0
4,98,49,30.0


In [50]:
rmse = np.sqrt(np.sum((train_product_count.loc[:,'products_count']-train_product_count.loc[:,'num_products'])**2)/train_product_count.shape[0])
display(rmse)

6.007396483106449

In [54]:
train_product_count.to_pickle('train_product_count.pickle')